## 1. Podstawy - propagacja wsteczna, metoda gradientu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Stwórzmy jakiś zbiór, którego nie da się łatwo "uchwycić" metodami liniowymi. Najprostszym pomysłem są okręgi o różnych promieniach, i klasie zależnej od promienia.

In [ ]:
n_classes = 3
points_per_class = 200
X = np.zeros((points_per_class*n_classes, 2))
Y = np.zeros(points_per_class*n_classes, dtype=int)
for i in range(n_classes):
    idx = range(points_per_class*i,points_per_class*(i+1))
    
    r = i + np.random.random(points_per_class)* 0.5 # 
    t = np.random.random(points_per_class) *2* np.pi
    
    X[idx] = np.c_[r*np.sin(t), r*np.cos(t)]
    Y[idx] = int(i)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=Y, s=40, cmap=plt.cm.Spectral)
plt.show()

Pierwszą próbę klasyfikacji zrobimy za pomocą prostego klasyfikatora liniowego, to znaczy bezpośrednio po warstwie wejściowej mamy 3-elementową warstwę wyjściową reprezentującą prawdopobieństwo przynależności do danej klasy.

Postać macierzowa:
$$ y = f(U),$$
$$ U = XW +b $$

gdzie $f$ jest funkcją softmax:,

$$ \hat{y_j} = \sigma (u)_j = \frac{\exp{u_j}}{\sum_{j = 1}^{k} \exp{u_j}} $$

dla $j = 1, ..., k$.

Będziemy optymalizować funkcję straty 
$$ L(y, \hat{y}) = -\frac{1}{N} \sum_{n} y_n\log{\hat{y_n}}  $$

Przydatna rzecz: 
$$ \frac{\partial L}{\partial u_j} = \hat{y_j} - y_j $$

In [ ]:
# randomizacja wartości początkowych
D = 2
K = 3
W = 1 * np.random.randn(D,K)
b = np.zeros((1,K))

# parametry 
step_size = 1e-2
reg = 1e-4

num_examples = X.shape[0]

for i in range(200):  
    # obliczamy scory oraz przynależności do klas przy aktualnych wagach
    
    

    # obliczamy funkcję straty dla aktualnej predykcji
    
    

    # wyliczamy gradient funkcji straty
    
    
    
    # it's backpropagation time!
    
    

    # dodajemy wpływ regularyzacji do gradientu wag
    

    # aktualizujemy parametry
    

In [ ]:
# predykcja - postępujemy tak, jak na początku kroku uczenia
scores = np.dot(X, W) + b
predicted_class = np.argmax(scores, axis=1) # nie musimy wyliczać konkretnych prawdopodobienstw - exp jest funkcją rosnącą
print('overall accuracy: %.2f' % (np.mean(predicted_class == Y)))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=predicted_class, s=40, cmap=plt.cm.Spectral)
plt.show()

Klasyfikator z warstwą ukrytą

In [ ]:
# POKAZAĆ co się dzieje przy dodaniu wag początkowych na 0
# randomizacja wartości początkowych
D = 2
K = 3
h = 12 # size of hidden layer
W = 1 * np.random.randn(D,h) #np.zeros((D,K)) 
b = np.zeros((1,h))
W2 = 1 * np.random.randn(h,K) #np.zeros((D,K)) 
b2 = np.zeros((1,K))

# parametry 
step_size = 1e-0
reg = 5e-2 # regularization strength


num_examples = X.shape[0]
for i in range(30):  
    # obliczamy scory oraz przynależności do klas przy aktualnych wagach
    

    # obliczamy funkcję straty dla aktualnej predykcji
    

    # wyliczamy gradient funkcji straty
   

    # wsteczna propagacja gradientu
    # najpierw badamy wpływ wag i stałej w ostatniej warstwie
       
    # następnie liczymy gradient dla wartości w warstwie ukrytej
    
    # pamietamy o uwzględnieniu pochodnej funkcji aktywacyjnej - na szczęście jest dość prosta
    
    # na koniec dostajemy gradienty dla pierwszej warstwy
   

    # dodajemy jeszcze wpływ regularyzacji do gradientu wag    

    # aktualizujemy parametry

In [ ]:
hidden_layer = np.maximum(0, np.dot(X, W) + b)
scores = np.dot(hidden_layer, W2) + b2
predicted_class = np.argmax(scores, axis=1)
print('training accuracy: %.2f' % (np.mean(predicted_class == Y)))

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=predicted_class, s=40, cmap=plt.cm.Spectral)
plt.show()

## 2. Keras

In [ ]:
import keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, Flatten
from keras.optimizers import SGD, adam
from keras.utils.np_utils import to_categorical

#### 2.1 Kręgi w zbożu

Najpierw zbudujemy sieci analogiczne do poprzednich - dzięki temu zobaczymy jak działają podstawowe funkcje i klasy, oraz przekonamy się jak bardzo user-friendly jest keras ;)

Na pierwszy ogień - model liniowy.


In [ ]:
# 1. budowanie modelu - dodawanie kolejnych warstw
model_linear = Sequential()

In [ ]:
# 2. kompilacja modelu - określenie algorytmu optymalizacji, oraz funkcji straty

In [ ]:
# 3. 'fit' - czyli uczymy model na danych treningowych 


In [ ]:
preds_linear = model_linear.predict(X)
print('training accuracy: %.2f' % (np.mean(np.argmax(preds_linear, axis=1) == Y)))

Następnie model z jedną warstwą ukrytą.

In [ ]:
# 1.
model_2l = Sequential()

# 2.

# 3.


In [ ]:
preds_2l = model_2l.predict(X)
print('training accuracy: %.2f' % (np.mean(np.argmax(preds_2l, axis=1) == Y)))

### Bostońskie domostwa

In [ ]:
from keras.datasets import boston_housing

In [ ]:
(X_train_boston, Y_train_boston), (X_test_boston, Y_tes_boston) = boston_housing.load_data()
print(X_train.shape)

In [ ]:
# 1.
model_boston_linear = Sequential()

# 2.

# 3.


In [ ]:
def describe_column(col):
    return( (np.min(col), np.max(col), np.mean(col)))

In [ ]:
stats = np.apply_along_axis(describe_column, 0, X_train_boston)
stats.T

In [ ]:
X_train_boston /= stats[1]
X_test_boston /= stats[1]

In [ ]:
# 1.
model_boston_linear = Sequential()

# 2.

# 3.


In [ ]:
# obliczenie funkcji straty na zbiorze testowym
np.mean((model_boston_linear.predict(X_test_boston).reshape(Y_test.shape_boston) - Y_test_boston)**2)

In [ ]:

# 1.
model_boston_2l = Sequential()

# 2.

# 3.



In [ ]:
# obliczenie funkcji straty na zbiorze testowym
np.mean((model_boston_2l.predict(X_test_boston).reshape(Y_test_boston.shape) - Y_test_boston)**2)

### Rozpoznawanie pisma

In [ ]:
from keras.datasets import mnist

In [ ]:
(X_train_mnist, Y_train_mnist), (X_test_mnist, Y_test_mnist) = mnist.load_data()
print(X_train_mnist.shape)
print(Y_train_mnist.shape)

In [ ]:
digit = X_train_mnist[0]
plt.imshow(digit, interpolation = "nearest", cmap = "gray")
plt.show()

In [ ]:
# 1.
model_boston_2l = Sequential()

# 2.

# 3.



In [ ]:
preds_mnist = model_mnist.predict(X_test_mnist)
print('test accuracy: %.2f' % (np.mean(np.argmax(preds_mnist, axis=1) == Y_test_mnist)))